In [38]:
"""
读取高德给的aoi数据orig_data/buildings.csv
去除面积过小的或内部包含了其它aoi的aoi
调用百度地图API获得楼名
"""
from tqdm import tqdm
from pprint import pprint
import requests
import pickle
import folium
import random
import pandas as pd
import matplotlib.pyplot as plt
from pyproj import Proj
from shapely.geometry import Polygon
from eviltransform import gcj2wgs, bd2wgs, bd2gcj, wgs2gcj, wgs2bd
random.seed(233)
projector = Proj("+proj=utm +zone=50 +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
regions = pickle.load(open("data/regions_all.pkl", "rb"))
WGS = 0
GCJ = 1
BD = 2

In [1]:
user_agents = [
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60",
    "Opera/8.0 (Windows NT 5.1; U; en)",
    "Mozilla/5.0 (Windows NT 5.1; U; en; rv:1.8.1) Gecko/20061208 Firefox/2.0.0 Opera 9.50",
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; en) Opera 9.50",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0",
    "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.71 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.133 ",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.101 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/537.75.14",
    "Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Win64; x64; Trident/6.0)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11",
    "Opera/9.25 (Windows NT 5.1; U; en)",
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12",
    "Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9",
    "Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7",
    "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0 ",
]

In [35]:
buildings = []
for _, poly, _ in pd.read_csv("orig_data/buildings.csv").values:
    poly = poly[10:-2]
    points = [x.split(" ") for x in poly.split(",")]
    points = [[xx for xx in x if xx] for x in points]
    points = [(float(lon), float(lat)) for lon, lat in points]
    if points[-1] != points[0]:
        points.append(points[0])
    buildings.append([gcj2wgs(*p[::-1])[::-1] for p in points])
print("orig aois:", len(buildings))

tmp = []
for i, points in enumerate(buildings):
    poly = Polygon([projector(*p) for p in points])
    if poly.area > 40:  # 去除面积很小的aoi
        gate_xy = poly.centroid
        if not gate_xy.within(poly):
            gate_xy = poly.representative_point()
        gate_xy = gate_xy.coords[:][0]
        gate_gps = projector(*gate_xy, inverse=True)
        lon, lat = gate_gps
        if lon < 116.41891275128775:
            task_id = "金辉"
        elif lat < 39.95802524813388:
            task_id = "锦炜"
        else:
            task_id = "复旦"
        tmp.append({
            "id": i,
            # "name": name,
            "points": points,
            "poly": poly,
            # "is_elevator": is_elevator,
            "gate_gps": gate_gps,
            "gate_xy": gate_xy,
            "gate_id": str(i),
            "task_id": task_id
        })
buildings = tmp
print("after removing small buildings:", len(buildings))

all_polys = []
is_parent = []
for i, bd1 in enumerate(buildings):
    poly1 = bd1["poly"]
    for j, bd2 in enumerate(buildings):
        if j == i:
            continue
        poly2 = bd2["poly"]
        # if poly2.within(poly1):
        #     is_parent.append(i)
        #     break
        if poly2.intersection(poly1).area > poly2.area * 0.7:
            is_parent.append(i)
            break
buildings = [bd for i, bd in enumerate(buildings) if i not in set(is_parent)]
print("after removing parent buildings:", len(buildings))


orig aois: 904
after removing small buildings: 873
after removing parent buildings: 733


In [36]:
# 百度API
user_key = "9vHb8Vf7LhlWeqh4vuFCafFB9f9dST5t"
radius = 50  # 搜索半径
url_base_nearby_search = f"https://api.map.baidu.com/place/v2/search?&query=住宅&radius={radius}&output=json&ak={user_key}&scope=2&filter=sort_name:distance|sort_rule:1"
cnt = 0
for bd in tqdm(buildings):
    lon, lat = bd["gate_gps"]
    lat, lon = wgs2bd(lat, lon)
    t = requests.get(url_base_nearby_search + f"&location={lat},{lon}").json()["results"]
    if t:
        info = t[0]  # results按distance排序, 取最近的作为该aoi的信息
        name = info["name"]
        cnt += 1
    else:
        name = "未知"
    bd["name"] = name
print("buildings with name:", cnt)


100%|██████████| 733/733 [01:27<00:00,  8.40it/s]

buildings with name: 607


In [37]:
for bd in buildings:
    del bd["poly"]
pickle.dump(buildings, open("data/buildings_all.pkl", "wb"))

In [39]:
buildings = pickle.load(open("data/buildings_all.pkl", "rb"))

In [40]:
pprint(buildings[0])

{'gate_gps': (116.41413861354422, 39.956938863260746),
 'gate_id': '0',
 'gate_xy': (449959.4945745536, 4423142.176433192),
 'id': 0,
 'name': '和平里二区-1号楼',
 'points': [(116.41415407487821, 39.956970990937954),
            (116.4142081337361, 39.95697103410792),
            (116.41421114479814, 39.95691003891668),
            (116.414068990864, 39.956905925814496),
            (116.41406597982595, 39.956966921024836),
            (116.41415407487821, 39.956970990937954)],
 'task_id': '金辉'}


In [41]:
G_folium = folium.Map(
    location=[39.967691, 116.426234],
    control_scale=True,
    tiles='http://webrd02.is.autonavi.com/appmaptile?lang=zh_cn&size=1&scale=1&style=8&x={x}&y={y}&z={z}',
    attr='高德底图',
    zoom_start=20,
)
for bd in buildings:
    if bd["task_id"] == "复旦":
        color = "black"
    elif bd["task_id"] == "金辉":
        color = "orange"
    elif bd["task_id"] == "锦炜":
        color = "royalblue"
    folium.PolyLine(
        locations=[wgs2gcj(*p[::-1]) for p in bd["points"]],
        opacity=0.8,
        weight=3,
        popup=(bd["id"], bd["name"]),
        color=color
    ).add_to(G_folium)
    folium.CircleMarker(
        location=wgs2gcj(*bd["gate_gps"][::-1]),
        opacity=0.8,
        radius=2,
        fill=True,   
        popup=(bd["gate_id"], bd["name"]),
        color=color
    ).add_to(G_folium)
for r in regions:
    folium.PolyLine(
        locations=[wgs2gcj(*p[::-1]) for p in r["boundary"]],
        opacity=0.8,
        weight=2,
        popup=r["rid"],
        color="red",
    ).add_to(G_folium)
G_folium.save("figure/buildings_all.html")